In [ ]:
# Create a subset of the mgf file for testing
with open("./massivekb_82c0124b.mgf", 'r') as infile, open("./massivekb_subset.mgf", 'w') as outfile:
    for i, line in enumerate(infile):
        if i >= 2e7:
            break
        outfile.write(line)

In [ ]:
import re
from time import time

import numpy as np
from pyteomics import mgf
from tqdm import tqdm

massivekb_file = "./massivekb_82c0124b.mgf"

CAPS_ONLY = re.compile(r'[^A-Z]')


def remove_ptms(seq):
    return CAPS_ONLY.sub('', seq)


t = time()
nones = 0
seqs = []
with mgf.read(massivekb_file, use_index=False, convert_arrays=0, read_charges=False, read_ions=False, ) as massivekb:
    for i, spectrum in enumerate(tqdm(massivekb, total=None)):
        try:
            seqs.append(remove_ptms(spectrum['params']['seq']))
        except TypeError as e:
            if spectrum is None:
                nones += 1
                continue
            else:
                raise e

print(f"Reading {len(seqs)} sequences from {massivekb_file} took {(time() - t) / 60:.2f} minutes")
print(f"{nones} invalid spectra")
print(f"{len(set(seqs))} unique sequences")

np.save("massivekb_82c0124b_seqs.npy", set(seqs), allow_pickle=True)

In [ ]:
import numpy as np

# Load the set of unique peptide sequences as a list
seqs = list(np.load("massivekb_82c0124b_seqs.npy", allow_pickle=True).item())

In [ ]:
from multiprocessing import Pool, cpu_count
import Levenshtein
from tqdm import tqdm
import random

n_datasail = 100000
norm_method = "max"
random_seqs = random.sample(seqs, n_datasail)

dist_matrix = np.zeros((n_datasail, n_datasail), dtype=np.uint8)


def compute_row(i):
    row = np.zeros(n_datasail - i - 1, dtype=np.uint8)  # only store distances for j > i
    for offset, j in enumerate(range(i + 1, n_datasail)):
        d = Levenshtein.distance(random_seqs[i], random_seqs[j])
        if norm_method == "max":
            d /= max(len(random_seqs[i]), len(random_seqs[j]))
            d *= 100
        row[offset] = d
    return i, row


with Pool(cpu_count()) as pool:
    # Parallelize over rows i
    for i, row in tqdm(pool.imap_unordered(compute_row, range(n_datasail - 1)), total=n_datasail - 1):
        dist_matrix[i, i + 1:] = row
        dist_matrix[i + 1:, i] = row

np.save(f"massivekb_dist_matrix_{norm_method}_{n_datasail}.npy", dist_matrix)
np.save(f"massivekb_seqs_{n_datasail}.npy", random_seqs, allow_pickle=True)
dist_matrix


In [ ]:
import pandas as pd
import numpy as np
from datasail.sail import datasail

n_datasail = 100000
norm_method = "max"

dist_matrix = np.load(f"massivekb_dist_matrix_{norm_method}_{n_datasail}.npy")
random_seqs = np.load(f"massivekb_seqs_{n_datasail}.npy", allow_pickle=True)

e_splits, _, _ = datasail(
    verbose='D',
    techniques=["C1e"],
    splits=[.84, .08, .08],
    names=["train", "val", "test"],
    runs=1,
    e_type="O",
    epsilon=0.1,
    e_clusters=200,
    e_data={i: s for i, s in enumerate(random_seqs)},
    e_dist=(list(range(n_datasail)), dist_matrix),
    threads=64,
)

df = pd.DataFrame.from_dict(e_splits['C1e'][0], orient='index', columns=['split'])
df['seq'] = random_seqs
df.to_csv(f"massivekb_split_df_{norm_method}_{n_datasail}.csv")

train_df = df[df['split'] == 'train']
val_df = df[df['split'] == 'val']
test_df = df[df['split'] == 'test']
print(len(train_df))
print(len(val_df))
print(len(test_df))

In [ ]:
from multiprocessing import Pool, cpu_count
from Levenshtein import distance as levenshtein_distance

# Global variables for use in the worker function
_global_seqs1 = None
_global_seqs2 = None


def _init_pool(seqs1, seqs2):
    global _global_seqs1, _global_seqs2
    _global_seqs1 = seqs1
    _global_seqs2 = seqs2


def compute_row(i):
    s1 = _global_seqs1[i]
    row = np.zeros(len(_global_seqs2), dtype=np.uint8)
    for j, s2 in enumerate(_global_seqs2):
        row[j] = levenshtein_distance(s1, s2)
    return (i, row)


def compute_distance_matrix(seqs1, seqs2):
    """
    Compute Levenshtein distance matrix between two lists of sequences in parallel.

    Args:
        seqs1 (List[str]): First list of sequences (rows).
        seqs2 (List[str]): Second list of sequences (columns).

    Returns:
        np.ndarray: Distance matrix of shape (len(seqs1), len(seqs2)) with dtype=uint8.
    """
    n1 = len(seqs1)
    n2 = len(seqs2)
    dist_matrix = np.zeros((n1, n2), dtype=np.uint8)

    # Create the pool and initialize globals
    with Pool(cpu_count(), initializer=_init_pool, initargs=(seqs1, seqs2)) as pool:
        for i, row in tqdm(pool.imap_unordered(compute_row, range(n1)), total=n1):
            dist_matrix[i, :] = row

    return dist_matrix

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

n_datasail = 100000
norm_method = "max"

df = pd.read_csv(f"massivekb_split_df_{norm_method}_{n_datasail}.csv")

train_df = df[df['split'] == 'train']
val_df = df[df['split'] == 'val']
test_df = df[df['split'] == 'test']
print(len(train_df))
print(len(val_df))
print(len(test_df))

train_lengths = train_df['seq'].str.len()
plt.hist(train_lengths)
plt.show()

val_lengths = val_df['seq'].str.len()
plt.hist(val_lengths)
plt.show()

test_lengths = test_df['seq'].str.len()
plt.hist(test_lengths)
plt.show()

all_seqs = list(np.load("massivekb_82c0124b_seqs.npy", allow_pickle=True).item())

dist_matrix = compute_distance_matrix(all_seqs, df['seq'].values)
nearest_indices = np.argmin(dist_matrix, axis=1)
split_labels = df['split'].values[nearest_indices]

split_df = pd.DataFrame({'seq': all_seqs, 'split': split_labels})
split_df.to_csv(f"massivekb_full_split_df_{norm_method}_{n_datasail}.csv")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

n_datasail = 100000
norm_method = "max"

split_df = pd.read_csv(f"massivekb_full_split_df_{norm_method}_{n_datasail}.csv")

train_df = split_df[split_df['split'] == 'train']
val_df = split_df[split_df['split'] == 'val']
test_df = split_df[split_df['split'] == 'test']
print(len(train_df))
print(len(val_df))
print(len(test_df))

train_lengths = train_df['seq'].str.len()
plt.hist(train_lengths)
plt.show()

val_lengths = val_df['seq'].str.len()
plt.hist(val_lengths)
plt.show()

test_lengths = test_df['seq'].str.len()
plt.hist(test_lengths)
plt.show()

In [ ]:
from tqdm import tqdm
import re
import os
from pyteomics import mgf
import pandas as pd
import numpy as np
import random
from collections import defaultdict, Counter
import matplotlib.pyplot as plt


def remove_ptms(seq):
    return ''.join([c for c in seq if 'A' <= c <= 'Z'])


n_datasail = 100000
norm_method = "max"

n_train_peps = [int(i) for i in [1e5, 2.5e5, 5e5, 7.5e5, 1e6]]
# n_train_peps = [int(i) for i in [1e3, 1e4, 1e5]]

n_train_spectra = [1, 2, 5, 10, 20]

massivekb_dir = "/mnt/data/cdens/casanovo-scaling/massivekb_data/"
massivekb_file = os.path.join(massivekb_dir, "massivekb_82c0124b.mgf")

scaling_dir = os.path.join(massivekb_dir, f"scaling_data_{norm_method}_{n_datasail}")
os.makedirs(scaling_dir, exist_ok=True)

split_df = pd.read_csv(f"massivekb_full_split_df_{norm_method}_{n_datasail}.csv")
split_dict = dict(zip(split_df['seq'], split_df['split']))  # convert to dict for more efficient lookup

spectra = {k: [] for k in ['train', 'val', 'test']}
train_index = defaultdict(list)
with mgf.read(massivekb_file, use_index=False, convert_arrays=0, read_charges=False, read_ions=False) as massivekb:
    for spectrum in tqdm(massivekb, total=3e7):
        unmod_pep = remove_ptms(spectrum['params']['seq'])
        split = split_dict[unmod_pep]
        spectra[split].append(spectrum)

        # Creating an index of the spectra per modified peptide in the train dataset
        if split == 'train':
            train_index[spectrum['params']['seq']].append(len(spectra['train']) - 1)

mgf.write(spectra['val'], os.path.join(scaling_dir, f"val.mgf"))
mgf.write(spectra['test'], os.path.join(scaling_dir, f"test.mgf"))

# SOME PTM STATS FROM THE TRAIN DATASET
extracted = []
for seq in train_index.keys():
    matches = re.findall(r'([A-Z]?)([+-]?\d+\.\d+)', seq)
    for residue, ptm in matches:
        if residue == "":
            residue = "N-term"  # fillna equivalent
        extracted.append((residue, ptm))
mod_count = Counter(extracted)
print("PTM counts:")
print(mod_count)

spectra_count_dict = {p: len(l) for p, l in train_index.items()}
print(f"Number of modified peptides: {len(spectra_count_dict)}")
plt.hist(spectra_count_dict.values(),
         bins=np.arange(min(spectra_count_dict.values()) - 0.5, max(spectra_count_dict.values()) + 0.5, 1))
plt.xlabel("Number of spectra")
plt.ylabel("Number of peptides")
plt.yscale("log")
plt.show()

for n_train_s in n_train_spectra:
    # Get all peptides with enough spectra
    p_with_enough_spectra = [p for p, c in spectra_count_dict.items() if c >= n_train_s]
    remaining_peptides = [p for p, c in spectra_count_dict.items() if c < n_train_s]

    print(f"There are {len(p_with_enough_spectra)} peptides with at least {n_train_s} spectra")

    for n_train_p in n_train_peps:
        print(f"Getting {n_train_s} spectra for {n_train_p} peptides")
        if len(spectra_count_dict.keys()) < n_train_p:
            train_spectra = []
            print(f"There are only {len(spectra_count_dict.keys())} peptides but requested {n_train_p}")

        elif len(p_with_enough_spectra) >= n_train_p:
            # Sample random peptides and spectra
            train_peptides = random.sample(p_with_enough_spectra, n_train_p)
            train_spectra = [spec_i for train_pep in train_peptides for spec_i in
                             random.sample(train_index[train_pep], n_train_s)]
            print(f"Sufficient peptides with sufficient spectra, number of spectra selected: {len(train_spectra)}")
        else:
            # Get all peptides with enough spectra and sample random spectra
            train_spectra = [spec_i for train_pep in p_with_enough_spectra for spec_i in
                             random.sample(train_index[train_pep], n_train_s)]
            print(f"Got {len(train_spectra)} spectra from peptides with enough spectra")

            # Count how many are missing
            n_train_p_to_add = n_train_p - len(p_with_enough_spectra)

            # Select random peptides to add
            train_p_to_add = random.sample(remaining_peptides, n_train_p_to_add)

            # Add all spectra for these random peptides
            train_spectra += [spec_i for train_pep in train_p_to_add for spec_i in train_index[train_pep]]
            print(f"Added all spectra from random peptides, selected {len(train_spectra)} spectra in total now")

        # Shuffle, then get spectra from indices and write to mgf
        random.shuffle(train_spectra)
        train_spectra = [spectra['train'][i] for i in train_spectra]
        mgf.write(train_spectra, os.path.join(scaling_dir, f"train_{n_train_s}s_{n_train_p}p.mgf"))
        print()

## Create a random subset of the validation data

In [5]:
from pyteomics import mgf
import random
import os
from tqdm import tqdm

def sample_fraction(iter, frac):
    for item in iter:
        if random.random() < frac:  # 25% chance
            yield item


val_file = "scaling_data_max_100000/val.mgf"
fraction = 0.25
out_file = os.path.join("scaling_data_max_100000", f"val_{fraction}.mgf")
with mgf.read(val_file, use_index=False, convert_arrays=0, read_charges=False, read_ions=False, ) as val:
    mgf.write(tqdm(sample_fraction(val, fraction), total=2e6), out_file)




 26%|██▌       | 517938/2000000.0 [06:17<17:59, 1372.36it/s] 


# Create a subset of the training data for batch size determining

In [1]:
from pyteomics import mgf
import random
import os
from tqdm import tqdm

def sample_fraction(iter, frac):
    for item in iter:
        if random.random() < frac:  # 25% chance
            yield item


file = "scaling_data_max_100000/train_1s_100000p.mgf"
fraction = 0.2
out_file = os.path.join("scaling_data_max_100000", f"bs_finder.mgf")
with mgf.read(file, use_index=False, convert_arrays=0, read_charges=False, read_ions=False, ) as f:
    mgf.write(sample_fraction(f, fraction), out_file)